# Transfer Learning

Teste para a implementação de uma rede neural convolucional que utiliza o processo de transfer learning para a classificação de pinturas.

Tutoriais seguidos:
- https://domino.ai/blog/transfer-learning-in-python#body__048c711080e1
- https://neptune.ai/blog/transfer-learning-guide-examples-for-images-and-text-in-keras

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import shutil

Utilizarei o dataset WikiArt, disponível nesse [link](https://www.kaggle.com/datasets/steubk/wikiart)

Para começar, é necessário separar o dataset em três partes:

- Treinamento: Utilizada para retreinar as camadas específicas do modelo
- Validação: Utilizada para ver se o treinamento do modelo está resultando em overfitting
- Teste: Avaliar a acurácia do modelo

In [5]:
def split_data(source_dir, train_dir, val_dir, test_dir, split_size=0.8):
    """
    Splits the data from source_dir into train_dir and val_dir based on split_size.
    """
    class_names = os.listdir(source_dir)
    
    for class_name in class_names:
        class_dir = os.path.join(source_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
            
        file_names = os.listdir(class_dir)
        file_paths = [os.path.join(class_dir, fname) for fname in file_names]
        
        train_paths_all, test_paths = train_test_split(file_paths, train_size=split_size, shuffle=True)
        
        train_paths, val_paths = train_test_split(train_paths_all, train_size=split_size, shuffle=True)
        
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
        
        for path in train_paths:
            shutil.copy(path, os.path.join(train_dir, class_name))
            
        for path in val_paths:
            shutil.copy(path, os.path.join(val_dir, class_name))
        
        for path in test_paths:
            shutil.copy(path, os.path.join(test_dir, class_name))

source_dir = "C:/Users/jalfr/OneDrive/Desktop/MASP/Code/Test/data/WikiArt/archive"
train_dir = "C:/Users/jalfr/OneDrive/Desktop/MASP/Code/Test/data/WikiArt/train"
val_dir = "C:/Users/jalfr/OneDrive/Desktop/MASP/Code/Test/data/WikiArt/val"
test_dir = "C:/Users/jalfr/OneDrive/Desktop/MASP/Code/Test/data/WikiArt/test"

split_data(source_dir, train_dir, val_dir, test_dir, split_size=0.8)

## Modelo - ResNet50

Import e utilização do modelo ResNet50, que será a base do projeto.

In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
model = ResNet50(weights='imagenet')
img_path = 'img/elephant.webp'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 38s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 6us/step 
Predicted: [('n02504458', 'African_elephant', 0.83670104), ('n01871265', 'tusker', 0.11563338), ('n02504013', 'Indian_elephant', 0.047628045)]
